### To which category belong a TO Term?

In [31]:
%%script bash
arq --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT distinct ?to ?label ?tcLabel
WHERE { 
    ?to rdfs:subClassOf* ?topCat .
    ?topCat rdfs:subClassOf <http://purl.obolibrary.org/obo/TO_0000387> .
    ?topCat rdfs:label ?tcLabel .
    ?to rdfs:label ?label 
} 
order by ?topCat
'

------------------------------------------------------------------------------------------------------------------------------
| to             | label                                                              | tcLabel                              |
| obo:TO_0002747 | "10-dehulled grain weight"                                         | "plant morphology trait"             |
| obo:TO_0000591 | "100-dehulled grain weight"                                        | "plant morphology trait"             |
| obo:TO_0000269 | "100-seed weight"                                                  | "plant morphology trait"             |
| obo:TO_0000592 | "1000-dehulled grain weight"                                       | "plant morphology trait"             |
| obo:TO_0000382 | "1000-seed weight"                                                 | "plant morphology trait"             |
| obo:TO_0000529 | "abaxial stomatal frequency"                                       | "plant morphology trait

### TO terms with more that 1 category

In [42]:
%%script bash
arq --results TSV --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  
        
SELECT ?to ?label (group_concat(distinct ?tcLabel ; separator = " AND ") AS ?topCatTot)
WHERE { 
    ?to rdfs:subClassOf* ?topCat .
    ?topCat rdfs:subClassOf <http://purl.obolibrary.org/obo/TO_0000387> .
    ?topCat rdfs:label ?tcLabel .
    ?to rdfs:label ?label 
} 
group by ?to
having (COUNT(?topCat) > 1)
order by ?tcLabel

'

Non-group key variable in SELECT: ?label
